In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from random import choice
from skimage import io
import sklearn.metrics

from tqdm.notebook import tqdm

In [ ]:
class CFG:
    BATCH_SIZE = 64
    EPOCHS = 250
    LEARNING_RATE = 3e-4

In [ ]:
TRAIN_PATH = '../input/dlcv-2021-hackathon/train/train/'
TEST_PATH = '../input/dlcv-2021-hackathon/test/'
sample_csv = pd.read_csv('../input/dlcv-2021-hackathon/sample_submission.csv')
test_csv = pd.read_csv('../input/dlcv-2021-hackathon/test.csv')

In [ ]:
# Storing File_Names

lst = os.listdir(TRAIN_PATH+'Post')
filenames = [file[5:] for file in lst]
print(len(filenames))

In [ ]:
class Custom_Dataset(Dataset):
   
    def __init__(self, TRAIN_PATH, filenames, transform=None):
        
        self.filenames = filenames
        self.post_op_path = TRAIN_PATH + "Post/"
        self.pre_op_path = TRAIN_PATH + "Pre/" 
        self.transform = transform
        df_columns = ['post_name', 'pre_name', 'label']
        self.df = pd.DataFrame(columns=df_columns)
        
        for i in range(len(filenames)):
            label = 1
            post_fn = "POST_"+filenames[i]
            pre_fn_true = "PRE_"+filenames[i]
            self.df = self.df.append(pd.Series([post_fn, pre_fn_true, label], index=self.df.columns), ignore_index=True)

            label=0
            rand_idx = choice([j for j in range(len(filenames)) if j not in [i]])
            pre_fn_false = "PRE_"+filenames[rand_idx]
            self.df = self.df.append(pd.Series([post_fn, pre_fn_false, label], index=self.df.columns), ignore_index=True)
            
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        post_name = row['post_name']
        pre_name = row['pre_name']
        label = row['label']
        
        post_img = Image.fromarray(io.imread(self.post_op_path+post_name))
        pre_img = Image.fromarray(io.imread(self.pre_op_path+pre_name))
        n, m = post_img.size

        post_img = post_img.crop((0, 0, n, int(m*3/4)))
        pre_img = pre_img.crop((0, 0, n, int(m*3/4)))
        
        
#         post_img = Image.fromarray(post_img.transpose(2,0,1))
#         pre_img = Image.fromarray(pre_img.transpose(2,0,1))        
#         post_img = np.array(Image.open(self.post_op_path+post_name),
#         pre_img = Image.open(self.pre_op_path+pre_name) 
        
        if self.transform:
#             print(type(post_img))
            post_img = self.transform(post_img)
            pre_img = self.transform(pre_img)
        
        return post_img, pre_img, label

In [ ]:
img1, img2 = test_csv.iloc[0].item().split(',')
img1 = img1.strip()
img2 = img2.strip()

print(img1, img2)
img1 = Image.fromarray(io.imread(TEST_PATH+img1))
img1

In [ ]:
class Test_Dataset(Dataset):
   
    def __init__(self, TEST_PATH, test_csv, transform=None):
        
        self.img_path = TEST_PATH
        self.transform = transform
        
    def __len__(self):
        return len(test_csv)
    
    
    def __getitem__(self, idx):
        img1_name, img2_name = test_csv.iloc[idx].item().split(',')
        img1_name = img1_name.strip()
        img2_name = img2_name.strip()
        
        img1 = Image.fromarray(io.imread(self.img_path+img1_name))
        img2 = Image.fromarray(io.imread(self.img_path+img2_name))
        
        n, m = img1.size

        img1 = img1.crop((0, 0, n, int(m*3/4)))
        img2 = img2.crop((0, 0, n, int(m*3/4)))
        
        if self.transform:
#             print(type(post_img))
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return img1, img2

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_size = 90
train_size = int((80/100) * len(filenames))
train_dataset = Custom_Dataset(TRAIN_PATH, filenames[:train_size], transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True)

val_dataset = Custom_Dataset(TRAIN_PATH, filenames[train_size:], transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE*2, shuffle=False)

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = Test_Dataset(TEST_PATH, test_csv, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=CFG.BATCH_SIZE*2, shuffle=False)

## DataLoader Sanity check

In [ ]:
post_img, pre_img, label = next(iter(train_dataloader))
print(post_img.shape)
post_img = post_img.numpy()
pre_img = pre_img.numpy()
show = 5


fig, ax = plt.subplots(5, 2, figsize=(14,14))
for i in range(5):
    ax[i][0].imshow(post_img[i].transpose(1,2,0))
    ax[i][0].set_title('Post Operation Label: '+str(label[i]))
    ax[i][0].axis('off')

    ax[i][1].imshow(pre_img[i].transpose(1,2,0))
    ax[i][1].set_title('Pre Operation label: '+str(label[i]))
    ax[i][1].axis('off')
    

plt.show()


In [ ]:
model_up = models.resnet18(pretrained=True)
model_dn = models.resnet18(pretrained=True)

In [ ]:
print(model_up)

In [ ]:
# FF_layer_up = [model_up.classifier[i] for i in [0,3,6]]
# FF_layer_dn = [model_dn.classifier[i] for i in [0,3,6]]

for model in [model_up, model_dn]:
    for params in model.parameters():
        params.requires_grad = False
        
# for i in [0,3,6]:
#     model_up.classifier[i] = FF_layer_up[i//3]
#     model_dn.classifier[i] = FF_layer_dn[i//3]    

# print(model_up)

# model_up.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5, inplace=False),
#     nn.Linear(in_features=4096, out_features=64, bias=True),
# #     nn.ReLU(inplace=True),
# #     nn.ReLU(inplace=True),
# #     nn.Dropout(p=0.5, inplace=False),
# #     nn.Linear(in_features=4096, out_features=1000, bias=True)
# )

# model_dn.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5, inplace=False),
#     nn.Linear(in_features=4096, out_features=64, bias=True),
# #     nn.ReLU(inplace=True),
# #     nn.ReLU(inplace=True),
# #     nn.Dropout(p=0.5, inplace=False),
# #     nn.Linear(in_features=4096, out_features=1000, bias=True)
# )

model_up.fc = nn.Linear(512,256)
model_dn.fc = nn.Linear(512,256)

print(model_up)

In [ ]:
class My_Model(nn.Module):
    def __init__(self, model_up, model_dn):
        super().__init__()
        self.model_up = model_up
        self.model_dn = model_dn
        
        self.linear1 = nn.Linear(512, 512)
        self.linear2 = nn.Linear(512, 2)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()
        
    def forward(self, post_img, pre_img):
        final_up = self.model_up(post_img)
#         print('final_up : ', final_up.shape)
        final_dn = self.model_dn(pre_img)   
#         print('final_dn : ', final_dn.shape)
        
        x = torch.cat((final_up, final_dn),1)
#         print(x.shape)
        x = self.relu(self.linear1(x))
#        x = self.dropout(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
        

In [ ]:
model = My_Model(model_up, model_dn)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG.LEARNING_RATE)

def train(model, optimizer, criterion):
    train_loss = []
    eval_loss = []
    macro_f1 = []
    for epoch in tqdm(range(CFG.EPOCHS)):
        train_loss_per_epoch = 0
        model.train()
        true_labels = []
        pred_labels = []
        
        for post_img, pre_img, labels in train_dataloader:
            post_img = post_img.to(device)
            pre_img = pre_img.to(device)
            labels = labels.to(device)
            out = model(post_img, pre_img)
            _, pred = torch.max(out, axis=1)
            
            loss = criterion(out, labels)

            train_loss_per_epoch += loss.item() / len(train_dataloader)
            
            true_labels.append(labels.cpu().numpy())
            pred_labels.append(pred.cpu().numpy())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        train_loss.append(train_loss_per_epoch)
        model.eval()
        with torch.no_grad():
            val_loss_per_epoch = 0
            for post_img, pre_img, labels in val_dataloader:
                post_img = post_img.to(device)
                pre_img = pre_img.to(device)
                labels = labels.to(device)
                out = model(post_img, pre_img)
                loss = criterion(out, labels)
                
                val_loss_per_epoch += loss.item() / len(val_dataloader)
                
        eval_loss.append(val_loss_per_epoch)
        
        true_labels = np.concatenate(true_labels)
        pred_labels = np.concatenate(pred_labels) 
        macro_f1_per_epoch = sklearn.metrics.f1_score(true_labels, pred_labels)
        macro_f1.append(macro_f1_per_epoch)
        
        print(f"EPOCH: {epoch+1}\tTRAIN_LOSS: {train_loss_per_epoch}\tVAL_LOSS: {val_loss_per_epoch}\tMACRO_F1: {macro_f1_per_epoch}")
        
    return train_loss, eval_loss, macro_f1
        
        

In [ ]:
train_loss, eval_loss, macro_f1 = train(model, optimizer, criterion)

In [ ]:
plt.plot(train_loss, label='Train')
plt.plot(eval_loss, label='Eval')
plt.show()

In [ ]:
plt.plot(macro_f1)
plt.show()

In [ ]:
scores = []

def inference(model):
    model.eval()
    with torch.no_grad():
        predictions = []
        for img1, img2 in tqdm(test_dataloader):
            img1 = img1.to(device)
            img2 = img2.to(device)
            out = model(img1, img2)
            scores.append(out.cpu().numpy())
            _, pred = torch.max(out, 1)
#             print(pred.cpu().numpy())
            predictions.append(pred.cpu().numpy())

    return predictions, scores


In [ ]:
predictions, scores = inference(model)

In [ ]:
test_pred = np.concatenate(predictions)
test_pred.shape

In [ ]:
tmp = test_csv.copy()
tmp[['0','1']] = np.concatenate(scores)
tmp.to_csv('Predictions_crop.csv', index=False)

In [ ]:
test_csv['Predicted'] = test_pred

In [ ]:
test_csv.to_csv('Submission.csv', index=False)

In [ ]:
import gc
gc.collect()